In [76]:
import os
import sys
root_dir = os.path.abspath('../')
sys.path.append(root_dir)

import seaborn as sns
import matplotlib.pyplot as plt

# --- Global style setup ---
sns.set_theme(style="white", context="notebook")

# Define a sequential green palette (dark → light)
green_palette = sns.color_palette("Greens", n_colors=6)  # can adjust n_colors if needed
sns.set_palette(green_palette)

plt.rcParams.update({
    "font.size": 12,
    "axes.labelweight": "semibold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "legend.fontsize": 11,
    "figure.facecolor": "white",
    "axes.edgecolor": "#333333",
    "axes.linewidth": 0.8,
    # optional: default line color order from dark to light green
    "axes.prop_cycle": plt.cycler("color", green_palette),
})

### Goal

We would like to model

$x_{i,t}=\Lambda f_t + \epsilon_{i,t}$

* $i$ = patient index
* $t$ = time (irregular visits per patient)
* $x_{i,t} \in R^p$ Patients embedding information (realization from the latent factor)
* $f_t \in R^r$ Shared latent temporatl factors (underlying population health states)
* $\Lambda \in R^{p\times r}$ Factor loadings (relationship between embeddings and latent factors)

### Mapping

| Model Element | Interpretation |
|---------------|----------------|
| Observed variables | Daily metrics (Counts by department, age group, diagnosis group) |
| Latent factors | Underlying patient-type intensities that jointly influence those metrics |
| Factor loadings | How strongly each variable with each latent patient type |
| Factor dynamics | How those patient types evolve over time (trends, cycles, shocks) |

In [2]:
import pandas as pd

# Dynamic Factor Model - engineered data
dfm_data = pd.read_parquet(
    os.path.join(root_dir, "data/processed/hana_ent/dfm_daily.parquet")
)
full_range = pd.date_range(dfm_data.index.min(), dfm_data.index.max(), freq='D')
dfm_data = dfm_data.reindex(full_range)
dfm_data = dfm_data.fillna(0)

# Supply data
supply = pd.read_parquet(
    os.path.join(root_dir, "./data/processed/hana_ent/supply.parquet")
)
full_range = pd.date_range(supply.index.min(), supply.index.max(), freq='D')
supply = supply.reindex(full_range)
supply = supply.fillna(0)

# Calendar characteristics


In [137]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor
import numpy as np
from tqdm import tqdm
import warnings
from joblib import Parallel, delayed
from sklearn.exceptions import ConvergenceWarning
from models.data_container.rolling_window_feeder import RollingWindowFeeder

warnings.filterwarnings('ignore', category=ConvergenceWarning)

def forecast_latent_factors(model, res, steps=14, freq="D"):
    k_factors = model.k_factors
    factor_order = model.factor_order
    transition_matrix = model.ssm['transition']
    
    # Get the last smoothed full state vector (includes lags if factor_order > 1)
    last_state = res.smoothed_state[:, -1]
    state = last_state.copy()
    
    # Forecast by iterating forward through the transition matrix
    states = []
    for _ in range(steps):
        state = transition_matrix @ state
        states.append(state.copy())

    states = np.vstack(states)
    factor_forecast = states[:, :k_factors]  # only the first k_factors are current-period factors

    # Build the forecast dataframe
    last_date = pd.to_datetime(res.data.dates[-1]) if res.data.dates is not None else pd.Timestamp.today()
    forecast_index = pd.date_range(last_date + pd.Timedelta(1, freq=freq), periods=steps, freq=freq)
    factor_forecast_df = pd.DataFrame(factor_forecast, index=forecast_index,
                                      columns=[f"Factor{i+1}" for i in range(k_factors)])
    
    return factor_forecast_df


def fit_and_predict_rf(y_col_name, X_train, y_train_col, X_test, y_true_col):
    """
    Fits a RandomForestRegressor for one supply series, predicts, and returns results.
    """
    model = RandomForestRegressor(
        n_estimators=500,
        max_depth=None,
        n_jobs=-1,  # use all cores internally
        random_state=42
    )
    model.fit(X_train, y_train_col)
    y_pred = model.predict(X_test)

    # Compute error metrics
    mae = np.mean(np.abs(y_pred - y_true_col))
    rmse = np.sqrt(np.mean((y_pred - y_true_col) ** 2))

    return {
        "supply": y_col_name,
        "model": model,
        "y_pred": y_pred,
        "y_true": y_true_col,
        "mae": mae,
        "rmse": rmse
    }


rw = RollingWindowFeeder(forward_window_days=14)  # 1 year -> 2 weeks
feeds = list(rw.feed(dfm_data.reset_index(), 'index'))
feeds = tqdm(feeds, total=len(feeds))
i = 0

for train_df, test_df, train_range, test_range in feeds:
    iter_dir = os.path.join(root_dir, "data/prediction/hana_ent/dfm_v1", f"iter_{i:03d}")
    os.makedirs(iter_dir, exist_ok=True)

    # Scale the model
    scaler = StandardScaler()
    scaled_train_df = scaler.fit_transform(train_df)
    scaled_test_df = scaler.transform(test_df)

    # Add noise to the model. - Current matrix is heavy-zeroed
    scaled_train_df += 1e-8 * np.random.randn(*scaled_train_df.shape)
    scaled_test_df += 1e-8 * np.random.randn(*scaled_test_df.shape)

    model = DynamicFactor(scaled_train_df, k_factors=3, factor_order=1, error_cov_type='diagonal')
    res = model.fit(maxiter=200, disp=False)

    # X: From train_df. Latent factors
    factors = pd.DataFrame(
        res.factors.smoothed.T,
        index=pd.date_range(train_range[0], train_range[-1], inclusive='left'),
        columns=[f"Factor{i+1}" for i in range(3)]
    )

    loadings = pd.DataFrame(
        res.params[:train_df.shape[1] * 3].reshape((train_df.shape[1], 3)),
        columns=[f"Loading{i+1}" for i in range(3)],
        index=train_df.columns
    )

    factors_predicted = forecast_latent_factors(model, res, steps=14)
    factors_predicted.index = pd.date_range(test_range[0], test_range[-1], freq='D', inclusive='left')

    # X, y
    y_train, y_true = (
        supply.loc[train_range[0] : train_range[-1]].iloc[:-1],
        supply.loc[test_range[0] : test_range[-1]].iloc[:-1]
    )

    X_train, X_test = (
        factors.values,
        factors_predicted.values
    )

    results = Parallel(n_jobs=-1, backend="loky")(
        delayed(fit_and_predict_rf)(
            col, X_train, y_train[col].values, X_test, y_true[col].values
        ) for col in y_train.columns
    )
    results_df = pd.DataFrame([{"supply": r["supply"], "MAE": r["mae"], "RMSE": r["rmse"]} for r in results])
    results_df = results_df.sort_values("RMSE")

    # Save artifacts
    factors.to_csv(os.path.join(iter_dir, "factors_train.csv"))
    factors_predicted.to_csv(os.path.join(iter_dir, "factors_forecast.csv"))
    loadings.to_csv(os.path.join(iter_dir, "factor_loadings.csv"))
    results_df.to_csv(os.path.join(iter_dir, "rf_results.csv"), index=False)

    # Plot latent factors (training + forecast)
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))
    for c in factors.columns:
        plt.plot(factors.index, factors[c], label=f"{c} (train)")
        plt.plot(factors_predicted.index, factors_predicted[c], linestyle="--", label=f"{c} (forecast)")
    plt.title(f"Latent Factors - Iteration {i}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(iter_dir, "latent_factors.png"))
    plt.close()

    # Save combined predictions
    pred_matrix = pd.DataFrame({r["supply"]: r["y_pred"] for r in results}, index=factors_predicted.index)
    true_matrix = y_true.copy()
    comparison_df = pd.concat({"Predicted": pred_matrix, "True": true_matrix}, axis=1)
    comparison_df.to_csv(os.path.join(iter_dir, "rf_predictions_vs_true.csv"))

    i += 1


  0%|          | 0/131 [00:00<?, ?it/s]/Users/goonzard/Developer/ds-202501-health-econ-mlflow/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  1%|          | 1/131 [00:29<1:04:12, 29.64s/it]/Users/goonzard/Developer/ds-202501-health-econ-mlflow/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  2%|▏         | 2/131 [00:57<1:01:10, 28.45s/it]/Users/goonzard/Developer/ds-202501-health-econ-mlflow/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  2%|▏         | 3/131 [01:27<1:02:47, 29.43s/i

ValueError: operands could not be broadcast together with shapes (14,) (5,) 

In [ ]:
# 결과 같음

import numpy as np
import pandas as pd

def forecast_latent_factors_from_model(res, steps=14, freq='D'):
    model = res.model
    kfilter = res.filter_results

    # transition이 3D인 경우 (factor_order = 0)
    if kfilter.transition.ndim == 3:
        T = kfilter.transition[:, :, -1]  # 마지막 slice 사용 (3D → 2D)
    else:
        T = kfilter.transition

    Q = kfilter.state_cov if kfilter.state_cov.ndim == 2 else kfilter.state_cov[:, :, -1]

    last_state = res.smoothed_state[:, -1]
    last_cov = res.smoothed_state_cov[:, :, -1]

    state_dim = last_state.shape[0]
    k_factors = model.k_factors

    states = np.zeros((steps, state_dim))
    covs = np.zeros((steps, state_dim, state_dim))

    for i in range(steps):
        next_state = T @ last_state
        next_cov = T @ last_cov @ T.T + Q

        states[i, :] = next_state
        covs[i, :, :] = next_cov

        last_state = next_state
        last_cov = next_cov

    # latent factor만 추출
    factor_forecast = states[:, :k_factors]

    last_date = (
        pd.to_datetime(res.data.dates[-1])
        if res.data.dates is not None
        else pd.Timestamp.today()
    )
    forecast_index = pd.date_range(
        last_date + pd.Timedelta(1, freq=freq), periods=steps, freq=freq
    )

    return pd.DataFrame(
        factor_forecast,
        index=forecast_index,
        columns=[f"Factor{i+1}" for i in range(k_factors)],
    )

factors_pred = forecast_latent_factors_from_model(res, steps=14)